In [1]:
import cv2
import numpy as np
import pdb

from Managers import *

# Motion Detection

In [2]:
ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,4))
kernel = np.ones((5, 5), dtype=np.uint8)
prev_frame = [None]

In [11]:
def trackMotionSimple(image, blur_size=21, min_size=1500, diff_thresh=25):
    background = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    background = cv2.GaussianBlur(background, (blur_size, blur_size), 0)
    
    if prev_frame[0] is None:
        prev_frame[0] = background
        return image
    
    diff = cv2.absdiff(prev_frame[0], background)
    _, diff = cv2.threshold(diff, diff_thresh, 255, cv2.THRESH_BINARY)
    diff = cv2.dilate(diff, ellipse, iterations=1)
    
    contours, hier = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if cv2.contourArea(c) > min_size:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
    diff = cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)
    prev_frame[0] = background
    return np.hstack((image, diff))

In [12]:
cam = CameraManager('Track Motion', cv2.VideoCapture(0), trackMotionSimple).addCloseCallback()
cam.run()